In [1]:
import harmbal as hb
import numpy as np
from pickle import load

t_py = []
t_f = []
t_hb = []
t_cupy = []

arr = np.arange(20, 150, 5)

for N in arr:

    print(N)
    #N = 30
    mass = 100
    L = 1

    d = L/N
    k = 8e4/d
    cp = 0.0002
    m = mass/N

    m_res = m / 2

    x0 = d/2
    w_res = 250
    N_res = 10 # number of resonators
    n_start = 5

    m_slope = False
    k_slope = False
    slope = np.linspace(0.7,1.3,N_res)

    beta = w_res**2*m_res/2
    alpha = beta/x0**2
    delta = 2 * cp * 2 * beta

    M = np.array(N*[m]+N_res*[m_res]) * np.eye(N+N_res)
    K = np.array(N*[2*k]+N_res*[0]) * np.eye(N+N_res)
    Snl = 0 * K

    for i in range(N-1):
        K[i, i + 1] = -k
        K[i + 1, i] = -k

    for i in range(N_res):
        dof = [n_start + i, N + i]
        k_el = np.array([[1, -1], [-1, 1]])
        # K[np.ix_(dof, dof)] += k_el * 2 * beta
        Snl[np.ix_(dof, dof)] += k_el

    C = cp * K + delta * Snl

    #M = np.array([[1, 0], [0, 2]])
    #K = np.array([[3, 0], [0, 0]])
    #Snl = np.array([[1, -1], [-1, 1]])

    #beta = 1**2*1/2
    #alpha = beta/x0**2

    sys = hb.Sys_NL(M=M, K=K, Snl=Snl, beta=-beta, alpha=alpha, n_harm=10, nu=1)


    sys.export_sys_data(p1={0:9+1.j, 1:8+2.j}, p2=64123782192678.2222)

    import datetime
    f = {0: 9e3}
    omg = 400
    t0 = datetime.datetime.now()
    try:
        x_hb, res = sys.solve_hb(f=f, omg=omg, full_output=True, state_space=True)
        print(datetime.datetime.now() - t0)
    except:
        print('Erro!')
        x_hb = np.zeros((sys.ndof * 2, 1))
    t_hb.append(datetime.datetime.now() - t0)
    # z = res[0]
    # fm = sys.floquet_multipliers(omg, z, dt_refine=250)
    # print(np.max(np.abs(fm)) > 1)

    t0 = datetime.datetime.now()

    dt = 0.5 * np.pi / np.max(np.imag(np.linalg.eig(sys.A_lin)[0]))
    tf = 30
    t_rk = np.arange(20000) * dt

    x0 = x_hb[:, 0].reshape((sys.ndof * 2, 1))
    t0 = datetime.datetime.now()
    x_rk4 = sys.solve_transient(f=f, omg=omg, t=t_rk, x0=x0, method='fortran', keep_data=True)
    print(datetime.datetime.now() - t0)
    t_f.append(datetime.datetime.now() - t0)
    t0 = datetime.datetime.now()
    x_rk42 = sys.solve_transient(f=f, omg=omg, t=t_rk, x0=x0, silent=True)
    print(datetime.datetime.now() - t0)
    t_py.append(datetime.datetime.now() - t0)
    t0 = datetime.datetime.now()
    x_rk42 = sys.solve_transient(f=f, omg=omg, t=t_rk, x0=x0, silent=True, cupy=True)
    print(datetime.datetime.now() - t0)
    
    t_cupy.append(datetime.datetime.now() - t0)
    
    
    
    print('---')
    
    


C:\Users\athom\anaconda3\lib\site-packages\cupy\_environment.py:205: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


20
0:00:01.062835
0:00:02.039557
0:00:02.456137
0:00:42.428815
---
25
0:00:01.450716
0:00:02.424042
0:00:02.590168
0:00:35.693992
---
30
0:00:01.759315
0:00:02.122348
0:00:02.162905
0:00:35.861520
---
35
0:00:02.466227
0:00:02.487870
0:00:02.267599
0:00:36.265920
---
40
0:00:03.288234
0:00:02.859748
0:00:02.450881


KeyboardInterrupt: 

In [6]:
import plotly.graph_objs as go
fig = go.Figure(data=[go.Scatter(x=t_rk,y=x_rk4[19,:]),
                      go.Scatter(x=t_rk,y=x_rk42[19,:])])

In [2]:
len(t_rk)

103617

In [14]:
t_rk[1]-t_rk[0]

0.0002895294562547952

In [16]:
t_rk[1]-t_rk[0]

0.0002895294562547952

In [54]:
fig = go.Figure(data=[go.Scatter(x=arr,y=[a.microseconds*1e-6+a.seconds for a in t_f], name='fortran'),
                      go.Scatter(x=arr,y=[a.microseconds*1e-6+a.seconds for a in t_py], name='Numpy'),
                     go.Scatter(x=arr,y=[a.microseconds*1e-6+a.seconds for a in t_hb], name='HB'),
                     go.Scatter(x=arr,y=[a.microseconds*1e-6+a.seconds for a in t_sp], name='SciPy')])
fig.write_html('comparacao metodos.html')
fig.write_image('comparacao metodos.png', scale=8)

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [45]:
fig = go.Figure(data=[go.Scatter(x=t_rk, y=x_rk4[19, :], name='fortran'),
                     go.Scatter(x=t_rk, y=x_rk42[19, :], name='numpy'),
                     go.Scatter(x=t_rk2, y=x_rk43, name='scipy')])

In [52]:
fig

In [14]:
t_f[0].total_seconds

<function timedelta.total_seconds>

In [10]:
t_f[0]

datetime.timedelta(seconds=2, microseconds=523499)

In [38]:
def mot_eq(t, y):
    
    y = y.reshape((sys.ndof * 2, 1))
    
    f = {0: 9e3}
    omg = 400
    
    F = np.zeros((2 * sys.ndof, 1))
    for n in f:
        F[n + sys.ndof, 0] = np.real(f[n]) * np.cos(omg * t) + np.imag(f[n]) * np.sin(omg * t)
    Z = np.zeros((sys.ndof, sys.ndof))
    Snl_stsp = np.vstack([np.hstack([Z, Z]),
                          np.hstack([-sys.Snl, Z])])
    v_aux = Snl_stsp @ y
    dy_dt = sys.A @ y + sys.Minv @ (F + sys.beta * v_aux + sys.alpha * v_aux ** 3)
    
    return dy_dt[:, 0]

from scipy.integrate import RK45
t0 = 0

res = RK45(mot_eq, t0=0, y0=x0[:,0], t_bound=t_rk[-1]+1)


t0 = datetime.datetime.now()

x_rk43 = []
t_rk2 = []
while res.t < t_rk[-1]:
    t_rk2.append(res.t)
    x_rk43.append(res.y[19])
    res.step()
    

print(datetime.datetime.now() - t0)

0:00:18.061635


In [33]:
res.

0

In [41]:
len(t_rk2)

10425

In [47]:
len(x_rk42[0,:])

20000

In [43]:
x_rk43[0]

-0.00019382607214221462